In [1]:
!pip install insightface onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.7 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp312-cp312-linux_x86_64.whl size=1071491 sha256=4e15109884fa90e3cfc1b0707b3f566af9404f7a59631c694b6198c8bea414b9
  Stored in directory: /root/.cache/pip/wheels/73/3c/e2/6d4815e8a8b33a2006554d65ce0d1f973e768f4c7a222fa675
Successfully built insightface


In [2]:
import cv2
import shutil
import numpy as np
import os
import zipfile
import tempfile
import random
import gc
from pathlib import Path
from tqdm import tqdm
from insightface.app import FaceAnalysis
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [5]:
ZIP_PATH = "/content/drive/MyDrive/Celeb-DF-v2.zip"
SAVE_BASE_PATH = "/content/drive/MyDrive/HECTO/Dataset/Celeb_frames"
NUM_FRAMES = 15
TARGET_SIZE = (256, 256)
FAKE_SAMPLE_COUNT = 890

In [6]:
# providers=['CUDAExecutionProvider']로 GPU 강제 사용
detector = FaceAnalysis(allowed_modules=['detection', 'landmark_2d'], providers=['CUDAExecutionProvider'])
detector.prepare(ctx_id=0, det_size=(640, 640))

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:03<00:00, 81513.56KB/s]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition
set det-size: (640, 640)


In [7]:
def get_hybrid_face(image, face_info, target_size=(256, 256)):
    try:
        h, w = image.shape[:2]
        bbox = face_info.bbox.astype(int)
        landmarks = getattr(face_info, 'kps', None)

        cx = (bbox[0] + bbox[2]) // 2
        cy = (bbox[1] + bbox[3]) // 2

        # 1️⃣ 정렬
        if landmarks is not None and len(landmarks) >= 2:
            left_eye, right_eye = landmarks[0], landmarks[1]
            dy, dx = right_eye[1] - left_eye[1], right_eye[0] - left_eye[0]
            angle = np.degrees(np.arctan2(dy, dx))
            M = cv2.getRotationMatrix2D((float(cx), float(cy)), angle, 1.0)
            image = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)

        # 2️⃣ 정사각 크롭
        side = int(max(bbox[2]-bbox[0], bbox[3]-bbox[1]) * 1.3)
        half = side // 2

        x1, y1 = cx - half, cy - half
        x2, y2 = cx + half, cy + half

        px1, py1 = max(0, -x1), max(0, -y1)
        px2, py2 = max(0, x2 - w), max(0, y2 - h)

        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)

        crop = image[y1:y2, x1:x2]

        if px1 or py1 or px2 or py2:
            crop = cv2.copyMakeBorder(
                crop, py1, py2, px1, px2,
                cv2.BORDER_CONSTANT, value=[0, 0, 0]
            )

        return cv2.resize(crop, target_size)

    except:
        return None

In [8]:
def run_preprocessing():
    # 저장 경로 초기화
    if os.path.exists(SAVE_BASE_PATH):
        shutil.rmtree(SAVE_BASE_PATH)
    os.makedirs(SAVE_BASE_PATH, exist_ok=True)

    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        all_videos = [
            f for f in z.namelist()
            if f.lower().endswith('.mp4') and not os.path.basename(f).startswith('._')
        ]

        # 데이터 필터링 및 샘플링
        real_videos = [f for f in all_videos if 'celeb-real' in f.lower() or 'youtube-real' in f.lower()]
        fake_all = [f for f in all_videos if 'celeb-synthesis' in f.lower()]
        fake_videos = random.sample(fake_all, min(len(fake_all), FAKE_SAMPLE_COUNT))

        print(f"📊 REAL: {len(real_videos)}, FAKE: {len(fake_videos)}")

        for video_list, label_type in [(real_videos, "REAL"), (fake_videos, "FAKE")]:
            print(f"\n🔥 {label_type} 처리 중...")

            for file_path in tqdm(video_list):
                video_name = Path(file_path).stem

                # 경로 설정 로직
                if 'youtube-real' in file_path.lower():
                    label_dir, person_id = 'real', 'youtube'
                else:
                    label_dir = 'real' if 'celeb-real' in file_path.lower() else 'fake'
                    person_id = video_name.split('_')[0]

                save_dir = os.path.join(SAVE_BASE_PATH, label_dir, person_id, video_name)
                os.makedirs(save_dir, exist_ok=True)

                with tempfile.NamedTemporaryFile(suffix=".mp4") as tmp:
                    tmp.write(z.read(file_path))
                    tmp.flush()

                    cap = cv2.VideoCapture(tmp.name)
                    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    if total <= 0:
                        cap.release()
                        continue

                    indices = np.linspace(0, total - 1, NUM_FRAMES, dtype=int)

                    face_crops = []
                    frames_cache = []      # 원본 프레임 보관 (소급 적용용)
                    last_face_info = None  # 🔥 이미지가 아닌 '위치 정보 객체'를 저장

                    for idx in indices:
                        cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
                        ret, frame = cap.read()
                        if not ret or frame is None:
                            continue

                        frames_cache.append(frame)
                        faces = detector.get(frame)
                        crop = None

                        if faces:
                            # 1. 얼굴 검출 성공 시
                            face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
                            crop = get_hybrid_face(frame, face) # 정렬 및 크롭
                            if crop is not None:
                                last_face_info = face # 🔥 좌표 정보 업데이트

                        elif last_face_info is not None:
                            # 2. 검출 실패 시 -> '이전 좌표'로 '현재 프레임'에서 크롭 (사용자 제안 로직)
                            crop = get_hybrid_face(frame, last_face_info)

                        if crop is not None:
                            # 🔥 처음 얼굴이 나온 경우, 그 전까지 비어있던 구간을 현재 좌표로 소급 적용
                            if len(face_crops) == 0 and len(frames_cache) > 1:
                                for i in range(len(frames_cache) - 1):
                                    retro_crop = get_hybrid_face(frames_cache[i], last_face_info)
                                    face_crops.append(retro_crop if retro_crop is not None else crop)

                            face_crops.append(crop)

                    cap.release()

                    # ===== 최종 저장 및 보정 =====
                    # 얼굴을 한 번도 못 찾은 경우에만 전체 리사이즈 수행
                    if len(face_crops) == 0:
                        for i, frame in enumerate(frames_cache[:NUM_FRAMES]):
                            resized = cv2.resize(frame, TARGET_SIZE)
                            cv2.imwrite(os.path.join(save_dir, f"frame_{i}.jpg"), resized)
                    else:
                        # 부족한 프레임은 마지막 크롭 이미지로 패딩
                        while len(face_crops) < NUM_FRAMES:
                            face_crops.append(face_crops[-1])

                        for i in range(NUM_FRAMES):
                            cv2.imwrite(os.path.join(save_dir, f"frame_{i}.jpg"), face_crops[i])

                gc.collect() # 메모리 정리

    print(f"\n✅ 완료: {SAVE_BASE_PATH}")

# 실행
run_preprocessing()

📊 REAL: 890, FAKE: 890

🔥 REAL 처리 중...


100%|██████████| 890/890 [1:53:12<00:00,  7.63s/it]



🔥 FAKE 처리 중...


100%|██████████| 890/890 [1:55:09<00:00,  7.76s/it]


✅ 완료: /content/drive/MyDrive/HECTO/Dataset/Celeb_frames
